# Ontology Tagger Tutorial
A hands-on notebook to configure BioPortal access, download ontology versions, and annotate terms with strict vs fallback behaviors.


## 1) Prerequisites
- Python 3.9+
- BioPortal account + API key
- Install deps: `pip install -r requirements.txt`
- Key files: `tagger.py`, `utils.py`, `const.py`, `main.py`
- Caching: OWL downloads saved under `downloads/{ontology}_{version}.owl`


## 2) Configure your BioPortal API key
1. Log in at https://bioportal.bioontology.org/accounts
2. Copy your API key from your account page.
3. Set it via environment or `.env` in repo root:
   - `export BIOPORTAL_API_KEY=your_key`
   - or create `.env` with `BIOPORTAL_API_KEY=your_key`


In [ ]:
# 3) Imports & client setup
from pathlib import Path

from tagger import BioPortalClient, OntologyTagger
from utils import get_api_key, outcomes_to_json

api_key = get_api_key()
client = BioPortalClient(api_key=api_key)
tagger = OntologyTagger(client)
print("Client ready; API key loaded.")


In [ ]:
# 4) Inspect available submissions (versions) for an ontology
subs = client.list_submissions("CL")
[(s.get("version"), s.get("submissionId")) for s in subs[:5]]  # show first few


## 5) Annotate with strict version (error if missing or absent)
- `on_version_missing="error"` is strict
- If the requested version is missing → raises `OntologyVersionNotFound`
- If the concept is not present in that version → returns strict outcome (no fallback)


In [ ]:
# Strict example
try:
    outcomes_strict = tagger.annotate_terms(
        ontology="CL",
        terms=["melanocyte"],
        version="2025-10-16",
        on_version_missing="error",
    )
    print(outcomes_to_json(outcomes_strict))
except Exception as e:
    print("Strict mode raised:", e)
finally:
    print("Download URL (if any):", client.last_download_url)


## 6) Annotate with flexible fallback
- `on_version_missing="latest"`
- If the requested version is missing OR the concept is absent in that version, it falls back to latest and notes the fallback in the comment.


In [ ]:
# Flexible example
outcomes_flex = tagger.annotate_terms(
    ontology="CL",
    terms=["melanocyte"],
    version="2025-10-16",
    on_version_missing="latest",
)
print(outcomes_to_json(outcomes_flex))
print("Download URL (latest or fallback):", client.last_download_url)


## 7) Annotate from a text file (one term per line)
Pass `file_path=Path("terms.txt")`. Empty lines are ignored.


In [ ]:
# File-based annotation
outcomes_file = tagger.annotate_terms(
    ontology="CL",
    file_path=Path("terms.txt"),
    version="2025-10-16",
    on_version_missing="latest",
)
print(outcomes_to_json(outcomes_file))


## 8) Notes on ontology IDs and matching
- Stored ontology IDs are short fragments (no full URIs).
- Presence in specified version is checked by ID and prefLabel in the downloaded OWL.
- If you rerun with the same version, the OWL file is reused from `downloads/` (cached).


## 9) Troubleshooting
- Restart kernel if you change code; stale imports can mask fixes.
- If an OWL file parses as JSON, ensure the URL has `/download` and the `apikey` query param (and Authorization header is set by the client).
- Inspect `client.last_download_url` to confirm the endpoint used.
- Ensure the requested version string matches a submission in `client.list_submissions()`.
